In [1]:
import pandas as pd
import numpy as np
from PY_Class_Def import clean_league_left_aut,clean_league_joined_aut

In [ ]:
df_fee=pd.read_csv(r"C:\Users\gandj\OneDrive\Desktop\Case_Study\Football\Transfers\Transfer_Target_Model\All_Trans_2000_2025.csv").iloc[:,1:]
df_fee.head()

,Player,Age,Club_Left,League_Left,Club_Joined,League_Joined,Transfer_Fee,Transfer_Window,Performance_Year
0,Luís Figo,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,60000000,2000,1999
1,Hernán Crespo,25,AC Parma,Serie A,Lazio Rom,Serie A,56810000,2000,1999
2,Gabriel Batistuta,31,AC Florenz,Serie A,AS Rom,Serie A,36150000,2000,1999
3,Nicolas Anelka,21,Real Madrid,LaLiga,FC Paris Saint-Germain,Division 1,34500000,2000,1999
4,Rio Ferdinand,22,West Ham United,Premier League,Leeds United,Premier League,26000000,2000,1999


In [9]:
df_player_stats=pd.read_csv(r"C:\Users\gandj\OneDrive\Desktop\Case_Study\Football\Transfers\Transfer_Target_Model\Player_Stats_2017_2025.csv").iloc[:,1:].rename(columns={'Main_Pos':'Pos'})
df_player_stats

,Player,Nation,Pos,Sec_Pos,Squad,Age,Born,MP,Starts,Min,90s,Gls,Gls_per_90,Ast,Ast_per_90,G+A,G+A_per_90,G-PK,G-PK_per_90,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,G+A-PK,G+A-PK_per_90,xG_per_90,xAG_per_90,xG+xAG,npxG_per_90,npxG+xAG_per_90,Total Tackels,TklW,Def 3rd,...,GA,GA90,SoTA,Saves,Save%,Penalty_Kick_Saves,W,D,L,CS,CS%,GK_PKatt,PKA,PKsv,PKm,FK,CK,OG,PSxG,PSxG/SoT,PSxG+/-,/90,Cmp,Launched_Att,Goal_Kick_Att,Cmp%,Att (GK),Thr,Launch%_Passes,Launch%_Goal_Kicks,AvgLen_Passes,AvgLen_Goal_Kicks,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist,League,Performance_Year
0,Aaron Cresswell,ENG,DF,DF,West Ham,34.0,1989.0,18,10,824,9.2,0,0.00,0,0.00,0,0.00,0,0.00,0,0,3,0,0.2,0.2,1.3,1.6,6,36,4,0.00,0.00,0.03,0.15,0.17,0.03,0.17,11,7,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,Premier League,2025
1,Aaron Ramsdale,ENG,GK,-,Southampton,26.0,1998.0,0,0,0,0.0,0,0.00,0,0.00,0,0.00,0,0.00,0,0,0,0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,...,66.0,2.2,188.0,120.0,67.6,28.6,2.0,5.0,23.0,3.0,10.0,7.0,5.0,2.0,0.0,0.0,10.0,0.0,63.5,0.32,-2.5,-0.08,196.0,605,190,32.4,1026.0,129.0,44.6,77.4,36.0,55.9,457.0,25.0,5.5,20.0,0.67,11.7,Premier League,2025
2,Aaron Ramsdale,ENG,GK,GK,Southampton,26.0,1998.0,30,30,2700,30.0,0,0.00,0,0.00,0,0.00,0,0.00,0,0,2,0,0.0,0.0,0.0,0.0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,Premier League,2025
3,Aaron Wan-Bissaka,ENG,DF,DF,West Ham,26.0,1997.0,36,35,3154,35.0,2,0.06,5,0.14,7,0.20,2,0.06,0,0,1,0,1.2,1.2,3.5,4.7,110,149,167,0.20,0.20,0.03,0.10,0.14,0.03,0.14,70,49,33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,Premier League,2025
4,Abdoulaye Doucouré,ML,MF,MF,Everton,31.0,1993.0,33,31,2564,28.5,3,0.11,2,0.07,5,0.18,3,0.11,0,0,5,1,4.1,4.1,2.4,6.6,42,82,96,0.18,0.18,0.15,0.09,0.23,0.15,0.23,48,28,21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,Premier League,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23647,Zaydou Youssouf,KM,FW,FW,Bordeaux,18.0,1999.0,6,1,131,1.5,0,0.00,0,0.00,0,0.00,0,0.00,0,0,2,0,0.1,0.1,0.0,0.2,7,4,9,0.00,0.00,0.08,0.03,0.11,0.08,0.11,3,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,Ligue 1,2018
23648,Zinédine Machach,FR,FW,MF,Toulouse,21.0,1996.0,4,2,193,2.1,1,0.47,0,0.00,1,0.47,1,0.47,0,0,2,0,0.2,0.2,0.3,0.5,6,7,14,0.47,0.47,0.09,0.16,0.25,0.09,0.25,7,3,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,Ligue 1,2018
23649,Ádám Lang,HU,DF,DF,Dijon,24.0,1993.0,3,3,232,2.6,0,0.00,0,0.00,0,0.00,0,0.00,0,0,1,1,0.0,0.0,0.0,0.0,0,7,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3,1,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,Ligue 1,2018
23650,Ángel Di María,AR,FW,FW,Paris S-G,29.0,1988.0,30,26,2111,23.5,11,0.47,6,0.26,17,0.72,11,0.47,0,0,1,0,12.1,12.1,7.8,19.9,95,128,287,0.72,0.72,0.51,0.33,0.85,0.51,0.85,13,10,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,Ligue 1,2018


In [10]:
#creating new age groups
def age_transformer(x):
    if x < 20 and x >= 16:
        return 'Teenager'
    elif x < 25 and x >= 20:
        return 'Early Twenties'
    elif x < 30 and x >= 25:
        return 'Late Twenties'
    elif x < 35 and x >=30:
        return 'Early Thirties'
    else:
        return 'Late Thirties'

#checking if a player was injured in a season
##x_row works as a placeholder for the dataframe
def limited_time(x_row):
    league = x_row['League']
    mp = x_row['MP']

    if league == 'Premier League' and mp < 38 * 0.35:
        return 1
    elif league == 'La Liga' and mp < 38 * 0.35:
        return 1
    elif league == 'Serie A' and mp < 38 * 0.35:
        return 1
    elif league == 'BundesLiga' and mp < 34 * 0.35:
        return 1
    elif league == 'Ligue 1' and mp < 34 * 0.35:
        return 1
    elif league == 'Série A Brasil' and mp < 38 * 0.35:
        return 1
    elif league == 'Eredivisie' and mp < 34 * 0.35:
        return 1
    elif league == 'Primeira Liga' and mp < 34 * 0.35:
        return 1
    elif league == 'Liga MX' and mp < 34 * 0.35:
        return 1
    elif league == 'Belgian Pro League' and mp < 34 * 0.35:
        return 1
    elif league == '2. BundesLiga' and mp < 34 * 0.35:
        return 1
    elif league == 'Ligue 2' and mp < 34 * 0.35:
        return 1
    elif league == 'Serie B' and mp < 38 * 0.35:
        return 1
    elif league == 'Segunda División' and mp < 42 * 0.35:
        return 1
    elif league == 'MLS' and mp < 34 * 0.35:
        return 1
    else:
        return 0

df_player_stats=pd.read_csv(r"C:\Users\gandj\OneDrive\Desktop\Case_Study\Football\Transfers\Transfer_Target_Model\Player_Stats_2017_2025.csv").iloc[:,1:].rename(columns={'Main_Pos':'Pos'})
df_player_stats=df_player_stats[(df_player_stats['Sec_Pos']!='GK')]

rev=list(df_player_stats.columns)
rev.remove('Sec_Pos')
df_player_stats=pd.read_csv(r"C:\Users\gandj\OneDrive\Desktop\Case_Study\Football\Transfers\Transfer_Target_Model\Player_Stats_2017_2025.csv").iloc[:,1:].rename(columns={'Main_Pos':'Pos'})
#df_player_stats=df_player_stats[(df_player_stats['Sec_Pos']!='-') & (df_player_stats['Pos']!='-')]
df_GK=df_player_stats[(df_player_stats['Pos']=='GK')].groupby(['Player','Nation','Pos','Squad','Age','Born','League','Performance_Year'])[list(df_player_stats)[7:-2]].sum().reset_index()
df_GK=df_GK[rev]

df_player_stats=pd.concat([df_player_stats,df_GK],axis=0)
#grouping the selected features and getting the average based on the grouping features
df_player_stats['Limited_Time']=df_player_stats.apply(limited_time,axis=1)
df_player_stats['Age_Num']=df_player_stats['Age'].copy()
df_player_stats['Age']=df_player_stats['Age'].apply(age_transformer)
df_player_stats.drop(['Sec_Pos'],axis=1,inplace=True)
group=list(df_player_stats.columns)[:5]
group.append(df_player_stats.columns[-4])
by=list(df_player_stats.columns)[6:-4]
for c in df_player_stats.columns[-3:]:
    by.append(c)
mean_player_stats=df_player_stats.groupby(group)[by].mean().reset_index()

group=list(df_player_stats.columns)[:5]
group.append(df_player_stats.columns[-4])
by=list(df_player_stats.columns)[-3]
max_year=df_player_stats.groupby(group)[by].max().to_frame().reset_index()

df_stats_new=pd.merge(mean_player_stats,max_year,on=list(max_year.columns)[:-1],how='left')
df_stats_new.drop(['Performance_Year_x'],axis=1,inplace=True)
df_stats_new.rename(columns={'Performance_Year_y':'Transfer_Window'},inplace=True)
df_stats_new['Pos'].replace({'MF,DF':'DF'},inplace=True)
df_stats_new
#data set with all leagues
df_stats_new.to_csv('Player_Stats_Mean_2017_2025.csv')

C:\Users\gandj\AppData\Local\Temp\ipykernel_30296\3253809579.py:84: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_stats_new['Pos'].replace({'MF,DF':'DF'},inplace=True)


In [11]:
df_stats_new

,Player,Nation,Pos,Squad,Age,League,MP,Starts,Min,90s,Gls,Gls_per_90,Ast,Ast_per_90,G+A,G+A_per_90,G-PK,G-PK_per_90,PK,PKatt,CrdY,CrdR,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,G+A-PK,G+A-PK_per_90,xG_per_90,xAG_per_90,xG+xAG,npxG_per_90,npxG+xAG_per_90,Total Tackels,TklW,Def 3rd,Mid 3rd,...,GA90,SoTA,Saves,Save%,Penalty_Kick_Saves,W,D,L,CS,CS%,GK_PKatt,PKA,PKsv,PKm,FK,CK,OG,PSxG,PSxG/SoT,PSxG+/-,/90,Cmp,Launched_Att,Goal_Kick_Att,Cmp%,Att (GK),Thr,Launch%_Passes,Launch%_Goal_Kicks,AvgLen_Passes,AvgLen_Goal_Kicks,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist,Limited_Time,Age_Num,Transfer_Window
0,Aaron Ciammaglichella,IT,MF,Torino,Teenager,Serie A,1.0,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,19.0,2025
1,Aaron Connolly,IE,FW,Brighton,Early Twenties,Premier League,10.5,5.0,473.500000,5.250000,1.000000,0.115000,0.500000,0.055,1.500000,0.170000,1.000000,0.115000,0.0,0.0,0.0,0.0,2.20,2.20,0.15,2.35,7.500000,2.500000,32.500000,0.170000,0.170000,0.330000,0.040000,0.370,0.330000,0.370,3.000000,1.5,0.500000,1.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,20.5,2022
2,Aaron Connolly,IE,FW,Brighton,Teenager,Premier League,24.0,14.0,1258.000000,14.000000,3.000000,0.210000,1.000000,0.070,4.000000,0.290000,3.000000,0.210000,0.0,0.0,0.0,0.0,4.00,4.00,0.50,4.50,21.000000,10.000000,86.000000,0.290000,0.290000,0.280000,0.040000,0.320,0.280000,0.320,7.000000,6.0,1.000000,4.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,2020
3,Aaron Cresswell,ENG,DF,West Ham,Early Thirties,Premier League,24.8,21.0,1878.200000,20.860000,0.400000,0.014000,2.400000,0.074,2.800000,0.088000,0.400000,0.014000,0.0,0.0,2.6,0.0,0.52,0.52,3.02,3.54,26.800000,111.000000,57.400000,0.088000,0.088000,0.022000,0.134000,0.154,0.022000,0.154,19.000000,10.2,10.200000,6.800000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,32.0,2025
4,Aaron Cresswell,ENG,DF,West Ham,Late Twenties,Premier League,29.0,28.0,2461.666667,27.366667,1.333333,0.043333,1.333333,0.050,2.666667,0.093333,1.333333,0.043333,0.0,0.0,5.0,0.0,0.90,0.90,2.50,3.30,37.666667,129.333333,87.333333,0.093333,0.093333,0.033333,0.086667,0.120,0.033333,0.120,35.333333,21.0,15.666667,16.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14394,Šime Vrsaljko,HR,FW,Atlético Madrid,Late Twenties,La Liga,9.0,6.0,519.000000,5.800000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2.0,0.0,0.00,0.00,0.00,0.00,13.000000,24.000000,39.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000,18.000000,12.0,13.000000,5.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,28.0,2021
14395,Žan Celar,SI,FW,Roma,Teenager,Serie A,1.0,0.0,5.000000,0.100000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.0

In [11]:
df_mean_stats=pd.read_csv(r"C:\Users\gandj\OneDrive\Desktop\Case_Study\Football\Transfers\Transfer_Target_Model\Player_Stats_Mean_2017_2025.csv").iloc[:,1:]

#transfer dataframe
df_trans=pd.read_csv(r"C:\Users\gandj\OneDrive\Desktop\Case_Study\Football\Transfers\Transfer_Target_Model\All_Trans_2000_2025.csv").iloc[:,1:]
df_trans.drop(['Age'],axis=1,inplace=True)
#appyling trans_value
#df_trans['Transfer_Fee']=df_trans['Transfer_Fee'].apply(trans_value)

df_stats=pd.merge(df_mean_stats,df_trans,on=['Player','Transfer_Window'],how='left').dropna()
#just to get actaull transfer fees
#df_stats=df_stats[df_stats['Transfer_Fee']!=0]
#df_stats['Pos'].replace({'MF,DF':'DF'},inplace=True)
df_stats['League'].replace({'La Liga':'LaLiga'},inplace=True)
df_stats['League_Left'].replace({'Vereinigte Staaten':'MLS','Liga NOS':'Liga Portugal'},inplace=True)
df_stats['League_Joined'].replace({'Vereinigte Staaten':'MLS','Liga NOS':'Liga Portugal'},inplace=True)
df_stats['League_Left'] = df_stats.apply(clean_league_left_aut, axis=1)
df_stats['League_Joined'] = df_stats.apply(clean_league_joined_aut, axis=1)
df_stats.to_csv('Final_Stats_2000_2025.csv')

C:\Users\gandj\AppData\Local\Temp\ipykernel_39072\3244681792.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_stats['League'].replace({'La Liga':'LaLiga'},inplace=True)
C:\Users\gandj\AppData\Local\Temp\ipykernel_39072\3244681792.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

Next, we will carry out the exploratory analysis. Here, we will check the distribution of the features and look for linearity and outliers. We will also check whether our target variable is balanced or imbalanced (i.e. whether one class is overrepresented).